In [1]:
import graphviz

import sys, os

os.environ["PATH"] = os.environ["PATH"] + ";" + r"C:\Program Files (x86)\Graphviz2.38\bin"

In [2]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import torch

import pyro
import pyro.infer
import pyro.optim
import pyro.distributions as dist
from itertools import product

In [3]:
%matplotlib inline

In [4]:
df_2_1 = pd.DataFrame([
    {"Y": 0, "X": 0, "Z": 0, "num": 8769 - 8173},
    {"Y": 1, "X": 0, "Z": 0, "num": 8173},
    
    {"Y": 0, "X": 0, "Z": 1, "num": 26231 - 19228},
    {"Y": 1, "X": 0, "Z": 1, "num": 19228},
    
    {"Y": 0, "X": 1, "Z": 0, "num": 26872 - 23339},
    {"Y": 1, "X": 1, "Z": 0, "num": 23339},
    
    {"Y": 0, "X": 1, "Z": 1, "num": 8128 - 5582},
    {"Y": 1, "X": 1, "Z": 1, "num": 5582}
])
df_2_1

df_2_1.index=[df_2_1.X, df_2_1.Z, df_2_1.Y]

nums = pd.Series(df_2_1.num, index=[df_2_1.X, df_2_1.Z, df_2_1.Y])

In [5]:
nums

X  Z  Y
0  0  0      596
      1     8173
   1  0     7003
      1    19228
1  0  0     3533
      1    23339
   1  0     2546
      1     5582
Name: num, dtype: int64

In [6]:
def model():
    Z = pyro.sample("Z", dist.Categorical(torch.tensor([nums.loc[:,0,:].sum() / nums.sum(), 
                                                        nums.loc[:,1,:].sum() / nums.sum()])))
    prob_X = torch.tensor([
        # Z = 0
        [nums.loc[0,0,:].sum() / nums.loc[:,0,:].sum(), nums.loc[1,0,:].sum() / nums.loc[:,0,:].sum()],
        # Z = 1
        [nums.loc[0,1,:].sum() / nums.loc[:,1,:].sum(), nums.loc[1,1,:].sum() / nums.loc[:,1,:].sum()]
    ])
    X = pyro.sample("X", dist.Categorical(prob_X[Z]))
    
    prob_Y = torch.tensor([
        # X = 0
        [ 
            # Z = 0
            [nums.loc[0,0,0] / nums.loc[0,0,:].sum(), nums.loc[0,0,1] / nums.loc[0,0,:].sum()],
            # Z = 1
            [nums.loc[0,1,0] / nums.loc[0,1,:].sum(), nums.loc[0,1,1] / nums.loc[0,1,:].sum()]
        ],
        # X = 1
        [
            # Z = 0
            [nums.loc[1,0,0] / nums.loc[1,0,:].sum(), nums.loc[1,0,1] / nums.loc[1,0,:].sum()],
            # Z = 1
            [nums.loc[1,1,0] / nums.loc[1,1,:].sum(), nums.loc[1,1,1] / nums.loc[1,1,:].sum()]
        ]
    ])
    Y = pyro.sample("Y", dist.Categorical(prob_Y[X, Z]))

In [7]:
trace_handler = pyro.poutine.trace(model)

## 3.1

In [8]:
def propensity_score(x, z):
    prob_X = torch.tensor([
        # Z = 0
        [nums.loc[0,0,:].sum() / nums.loc[:,0,:].sum(), nums.loc[1,0,:].sum() / nums.loc[:,0,:].sum()],
        # Z = 1
        [nums.loc[0,1,:].sum() / nums.loc[:,1,:].sum(), nums.loc[1,1,:].sum() / nums.loc[:,1,:].sum()]
    ])
    return prob_X[z][x]

### 3.2

In [19]:
NUM_SAMPLES = 1000
l_samples_unweighted = []
for i in range(NUM_SAMPLES):
    trace = trace_handler.get_trace()
    x = trace.nodes["X"]["value"].item()
    y = trace.nodes["Y"]["value"].item()
    z = trace.nodes["Z"]["value"].item()
    p = np.exp(trace.log_prob_sum())
    sample = (x, y, z, p)
    l_samples_unweighted.append(sample)

In [20]:
l_samples_unweighted[0:10]

[(0, 1, 0, tensor(0.1168)),
 (0, 1, 0, tensor(0.1168)),
 (1, 1, 0, tensor(0.3334)),
 (0, 1, 1, tensor(0.2747)),
 (0, 0, 1, tensor(0.1000)),
 (1, 1, 0, tensor(0.3334)),
 (1, 1, 0, tensor(0.3334)),
 (0, 1, 1, tensor(0.2747)),
 (1, 1, 0, tensor(0.3334)),
 (0, 1, 1, tensor(0.2747))]

#### Assemble an unweighted distibution
1. group the probabilities by combination of variables in dict

In [30]:
probs_unweighted = {s[0:3]: s[3] for s in l_samples_unweighted}

In [31]:
probs_unweighted

{(0, 1, 0): tensor(0.1168),
 (1, 1, 0): tensor(0.3334),
 (0, 1, 1): tensor(0.2747),
 (0, 0, 1): tensor(0.1000),
 (0, 0, 0): tensor(0.0085),
 (1, 0, 0): tensor(0.0505),
 (1, 1, 1): tensor(0.0797),
 (1, 0, 1): tensor(0.0364)}

In [32]:
# quick check that unweighted probabilities sum to 1
torch.sum(torch.tensor(list(probs_unweighted.values())))

tensor(1.0000)

2. now assemble sorted list of probabilities as input for categorical distribution

In [33]:
dist_unweighted_raw = [(k, v) for k,v in probs_unweighted.items()]

In [34]:
dist_unweighted_raw = sorted(dist_unweighted_raw)

In [35]:
dist_unweighted_raw

[((0, 0, 0), tensor(0.0085)),
 ((0, 0, 1), tensor(0.1000)),
 ((0, 1, 0), tensor(0.1168)),
 ((0, 1, 1), tensor(0.2747)),
 ((1, 0, 0), tensor(0.0505)),
 ((1, 0, 1), tensor(0.0364)),
 ((1, 1, 0), tensor(0.3334)),
 ((1, 1, 1), tensor(0.0797))]

3. weight with propensity scores

In [37]:
dist_weighted_raw = [(d[0], d[1]*(1.0 / propensity_score(d[0][0], d[0][2]))) for d in dist_unweighted_raw]

In [38]:
dist_weighted_raw

[((0, 0, 0), tensor(0.0346)),
 ((0, 0, 1), tensor(0.1310)),
 ((0, 1, 0), tensor(0.4746)),
 ((0, 1, 1), tensor(0.3598)),
 ((1, 0, 0), tensor(0.0669)),
 ((1, 0, 1), tensor(0.1538)),
 ((1, 1, 0), tensor(0.4422)),
 ((1, 1, 1), tensor(0.3371))]

4. re-normalize weighted probabilities

In [39]:
sum_probs_weighted = torch.tensor([d[1] for d in dist_weighted_raw]).sum()

In [41]:
dist_weighted_raw = [(d[0], d[1]*(1/sum_probs_weighted)) for d in dist_weighted_raw]

In [42]:
dist_weighted_raw

[((0, 0, 0), tensor(0.0173)),
 ((0, 0, 1), tensor(0.0655)),
 ((0, 1, 0), tensor(0.2373)),
 ((0, 1, 1), tensor(0.1799)),
 ((1, 0, 0), tensor(0.0335)),
 ((1, 0, 1), tensor(0.0769)),
 ((1, 1, 0), tensor(0.2211)),
 ((1, 1, 1), tensor(0.1685))]

In [43]:
dist_weighted = dist.Categorical(torch.tensor([e[1] for e in dist_weighted_raw]))

In [47]:
dist_weighted.probs.sum()

tensor(1.)

## 3.4

In [68]:
t_sample_weighted = dist_weighted.sample(sample_shape=torch.Size((1000,)))

In [69]:
t_sample_weighted[0:10]

tensor([2, 7, 3, 4, 3, 3, 6, 7, 7, 6])

In [96]:
l_sample_weighted_unpacked = [dist_weighted_raw[i][0] for i in t_sample_weighted]

In [97]:
l_sample_weighted_unpacked

[(0, 1, 0),
 (1, 1, 1),
 (0, 1, 1),
 (1, 0, 0),
 (0, 1, 1),
 (0, 1, 1),
 (1, 1, 0),
 (1, 1, 1),
 (1, 1, 1),
 (1, 1, 0),
 (1, 1, 0),
 (1, 1, 1),
 (1, 1, 0),
 (1, 1, 0),
 (0, 1, 0),
 (0, 1, 1),
 (1, 1, 1),
 (1, 1, 1),
 (1, 1, 1),
 (0, 1, 0),
 (1, 1, 1),
 (1, 1, 0),
 (0, 1, 0),
 (1, 1, 0),
 (1, 1, 0),
 (0, 1, 1),
 (0, 1, 0),
 (0, 1, 0),
 (1, 1, 0),
 (0, 0, 1),
 (0, 1, 1),
 (1, 1, 0),
 (0, 1, 0),
 (0, 1, 1),
 (1, 1, 0),
 (0, 1, 0),
 (1, 1, 0),
 (1, 1, 0),
 (0, 1, 1),
 (0, 1, 0),
 (1, 1, 0),
 (0, 1, 0),
 (1, 1, 0),
 (0, 1, 1),
 (1, 1, 1),
 (1, 1, 0),
 (0, 1, 0),
 (0, 1, 0),
 (1, 0, 1),
 (1, 1, 0),
 (0, 1, 0),
 (0, 0, 0),
 (1, 0, 1),
 (1, 1, 0),
 (0, 0, 1),
 (0, 1, 0),
 (0, 1, 0),
 (1, 0, 1),
 (0, 1, 0),
 (1, 0, 0),
 (1, 1, 1),
 (1, 1, 1),
 (0, 1, 1),
 (0, 1, 1),
 (0, 1, 0),
 (1, 1, 1),
 (1, 0, 1),
 (0, 1, 0),
 (1, 1, 1),
 (0, 1, 1),
 (0, 1, 0),
 (0, 1, 0),
 (1, 1, 1),
 (0, 1, 1),
 (0, 1, 0),
 (1, 1, 1),
 (0, 1, 1),
 (1, 1, 0),
 (1, 1, 0),
 (0, 1, 0),
 (1, 0, 1),
 (0, 1, 0),
 (0, 1, 0),
 (1,

In [100]:
t_sample_weighted_unpacked = torch.cat([torch.tensor([s]) for s in l_sample_weighted_unpacked])

In [101]:
t_sample_weighted_unpacked

tensor([[0, 1, 0],
        [1, 1, 1],
        [0, 1, 1],
        ...,
        [1, 1, 0],
        [1, 1, 0],
        [0, 0, 1]])

## 3.5



In [107]:
t_sample_weighted_unpacked.shape[0]

1000

In [134]:
def calc_prob_inv_prob_weighting(x, y):
    t_sample_weighted_filtered_x = t_sample_weighted_unpacked[t_sample_weighted_unpacked[:,0] == x]
    t_sample_weighted_filtered_x_y = t_sample_weighted_filtered_x[t_sample_weighted_filtered_x[:,1] == y]
    return float(t_sample_weighted_filtered_x_y.shape[0]) / float(t_sample_weighted_filtered_x.shape[0])

In [135]:
calc_prob_inv_prob_weighting(1, 1)

0.7755102040816326

In [136]:
calc_prob_inv_prob_weighting(0, 1)

0.807843137254902

In [137]:
def calc_prob_do(x, y, num_samples=1000):
    model_2_1_do = pyro.do(model, data={"X": torch.tensor(x)})
    imp_model_2_1_do = pyro.infer.Importance(model=model_2_1_do, num_samples=num_samples)
    inf_model_2_1_do = imp_model_2_1_do.run()
    return torch.exp(inf_model_2_1_do.marginal(["Y"]).empirical["Y"].log_prob(y))

In [138]:
calc_prob_do(1, 1)

tensor(0.7670)

In [139]:
calc_prob_do(0, 1)

tensor(0.8300)